# DATA MODELING - 2

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Paths
project_root = Path(os.getcwd()).parents[0]
data_folder = project_root / 'data' / 'data_accidents'
grid_weather_dir = data_folder / "grid_weather"
os.makedirs(grid_weather_dir, exist_ok=True)

In [5]:
# You'll need these later
try:
    import openmeteo_requests
    import requests_cache
    from retry_requests import retry
except ImportError:
    !pip install openmeteo-requests requests-cache retry-requests

# Setup Open-Meteo client (from your original code)
cache_session = requests_cache.CachedSession('.cache', expire_after=86400)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

In [8]:
### STEP 1: Check FARS columns across years (corrected for LONGITUD)
years_to_check = ["1980", "1990", "2000", "2010", "2020"]
for year in years_to_check:
    folder = data_folder / year
    acc_file = folder / 'accident.csv'
    if acc_file.exists():
        try:
            df = pd.read_csv(acc_file, nrows=1, encoding='utf-8')
        except:
            df = pd.read_csv(acc_file, nrows=1, encoding='latin1')
        print(f"\n{year} columns:")
        print(f"  Has LAT/LON: {'LATITUDE' in df.columns and 'LONGITUD' in df.columns}")
        print(f"  Has YEAR/MONTH/DAY: {all(c in df.columns for c in ['YEAR', 'MONTH', 'DAY'])}")
        print(f"  Has DATE: {'DATE' in df.columns}")
        if 'WEATHER' in df.columns:
            print(f"  Has WEATHER condition: True")


1980 columns:
  Has LAT/LON: False
  Has YEAR/MONTH/DAY: True
  Has DATE: False
  Has WEATHER condition: True

1990 columns:
  Has LAT/LON: False
  Has YEAR/MONTH/DAY: True
  Has DATE: False
  Has WEATHER condition: True

2000 columns:
  Has LAT/LON: True
  Has YEAR/MONTH/DAY: True
  Has DATE: False
  Has WEATHER condition: True

2010 columns:
  Has LAT/LON: True
  Has YEAR/MONTH/DAY: True
  Has DATE: False
  Has WEATHER condition: True

2020 columns:
  Has LAT/LON: True
  Has YEAR/MONTH/DAY: True
  Has DATE: False
  Has WEATHER condition: True


In [9]:
STATE_CENTROIDS = {
    1: (32.3182, -86.9023),   # Alabama
    2: (64.2008, -149.4853),  # Alaska
    4: (34.0489, -111.0937),  # Arizona
    5: (34.7996, -92.1849),   # Arkansas
    6: (37.2719, -119.2702),  # California
    8: (39.5501, -105.7821),  # Colorado
    9: (41.6032, -73.0877),   # Connecticut
    10: (39.1398, -75.5052),  # Delaware
    11: (38.8974, -77.0268),  # District of Columbia
    12: (27.6648, -81.5158),  # Florida
    13: (32.1656, -82.9001),  # Georgia
    15: (19.8968, -155.5828), # Hawaii
    16: (44.0682, -114.7420), # Idaho
    17: (39.7837, -89.6501),  # Illinois
    18: (39.7910, -86.1490),  # Indiana
    19: (41.8780, -93.0977),  # Iowa
    20: (38.5266, -96.7265),  # Kansas
    21: (37.8393, -85.7170),  # Kentucky
    22: (31.2448, -92.1450),  # Louisiana
    23: (45.2538, -69.4455),  # Maine
    24: (39.0458, -76.6413),  # Maryland
    25: (42.4072, -71.3824),  # Massachusetts
    26: (43.3266, -84.5361),  # Michigan
    27: (45.6945, -93.9002),  # Minnesota
    28: (32.3547, -90.0602),  # Mississippi
    29: (38.5443, -92.2884),  # Missouri
    30: (46.9653, -110.0845), # Montana
    31: (41.4925, -99.9018),  # Nebraska
    32: (38.8026, -116.4194), # Nevada
    33: (43.1939, -71.5724),  # New Hampshire
    34: (40.2237, -74.7647),  # New Jersey
    35: (34.5199, -106.0186), # New Mexico
    36: (42.7512, -75.7634),  # New York
    37: (35.7718, -80.0211),  # North Carolina
    38: (47.5515, -100.4659), # North Dakota
    39: (40.4173, -82.9071),  # Ohio
    40: (35.0078, -97.0929),  # Oklahoma
    41: (44.0009, -120.5542), # Oregon
    42: (40.5908, -77.2098),  # Pennsylvania
    44: (41.5801, -71.4774),  # Rhode Island
    45: (33.8361, -81.1637),  # South Carolina
    46: (44.5000, -100.0000), # South Dakota
    47: (35.5175, -86.5804),  # Tennessee
    48: (31.9686, -99.9018),  # Texas
    49: (40.1500, -111.8625), # Utah
    50: (44.5588, -72.5778),  # Vermont
    51: (37.4316, -78.6569),  # Virginia
    53: (47.7511, -120.7401), # Washington
    54: (38.5976, -80.4549),  # West Virginia
    55: (44.5000, -89.5000),  # Wisconsin
    56: (43.0000, -107.5000)  # Wyoming
}

This step constructs a unified **daily fatal crash dataset** that combines two spatial strategies based on FARS data availability:

- **1975–1999**: FARS does **not** include precise crash coordinates.
  → We assign each crash to its **state centroid** (using predefined latitude/longitude from `STATE_CENTROIDS`).

- **2000–2023**: FARS includes **actual crash coordinates** (`LATITUDE` and `LONGITUD`).
  → We use these exact locations (after filtering for valid U.S. coordinates: 24°N–50°N, 125°W–66°W).

#### Key Processing Steps:
1. **Parse date** from `YEAR`, `MONTH`, and `DAY` columns (excluding invalid dates like `DAY = 0`).
2. **Normalize longitude column name** from `LONGITUD` → `LONGITUDE` (FARS uses truncated SAS names).
3. **Filter coordinates** to remove zeros, nulls, and non-CONUS locations.
4. **Tag each record** with `location_type` (`'crash_exact'` or `'state_centroid'`) for transparency.
5. **Save the final dataset** as `fars_daily_hybrid_coords.csv`.

This hybrid approach ensures **maximal spatial accuracy** where possible while preserving **full historical coverage**, forming the foundation for high-resolution weather–crash analysis at the daily level.

In [10]:
### STEP 2: Build hybrid daily crash dataset (state-level before 2000, crash-level after)
def parse_hybrid_daily():
    all_crashes = []

    for year_folder in tqdm(sorted(data_folder.iterdir()), desc="Parsing FARS"):
        if not year_folder.is_dir():
            continue

        acc_file = year_folder / 'accident.csv'
        if not acc_file.exists():
            continue

        year = int(year_folder.name)

        try:
            df = pd.read_csv(acc_file, low_memory=False, encoding='utf-8')
        except:
            df = pd.read_csv(acc_file, low_memory=False, encoding='latin1')

        # Parse date
        if all(col in df.columns for col in ['YEAR', 'MONTH', 'DAY']):
            df = df[(df['DAY'] > 0) & (df['MONTH'] > 0)]
            df['date'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']], errors='coerce')
        else:
            continue

        if 'STATE' not in df.columns:
            continue

        if year >= 2000 and 'LATITUDE' in df.columns and 'LONGITUD' in df.columns:
            # 📍 Use actual crash coordinates (2000–2023)
            df = df.rename(columns={'LONGITUD': 'LONGITUDE'})
            df = df.dropna(subset=['LATITUDE', 'LONGITUDE'])
            df = df[(df['LATITUDE'] != 0) & (df['LONGITUDE'] != 0)]
            df = df[
                (df['LATITUDE'].between(24, 50)) &
                (df['LONGITUDE'].between(-125, -66))
                ]
            df['lat'] = df['LATITUDE']
            df['lon'] = df['LONGITUDE']
            df['location_type'] = 'crash_exact'

        else:
            # Use state centroid (1975–1999)
            df['lat'] = df['STATE'].map({k: v[0] for k, v in STATE_CENTROIDS.items()})
            df['lon'] = df['STATE'].map({k: v[1] for k, v in STATE_CENTROIDS.items()})
            df['location_type'] = 'state_centroid'

        # Keep one row per crash
        crashes = df[['date', 'STATE', 'lat', 'lon', 'location_type']].copy()
        crashes = crashes.dropna(subset=['date', 'lat', 'lon'])
        all_crashes.append(crashes)

    return pd.concat(all_crashes, ignore_index=True)


# Run it
df_crashes = parse_hybrid_daily()
print(f"Parsed {len(df_crashes):,} crashes")
print(f"Location types: {df_crashes['location_type'].value_counts().to_dict()}")

# Save
hybrid_file = data_folder / "fars_daily_hybrid_coords.csv"
df_crashes.to_csv(hybrid_file, index=False)

Parsing FARS: 100%|██████████| 60/60 [00:08<00:00,  7.01it/s]


Parsed 874,852 crashes
Location types: {'crash_exact': 532088, 'state_centroid': 342764}


In [11]:
df_crashes.head()

,date,STATE,lat,lon,location_type
0,1998-01-01,1,32.3182,-86.9023,state_centroid
1,1998-01-01,1,32.3182,-86.9023,state_centroid
2,1998-01-01,1,32.3182,-86.9023,state_centroid
3,1998-01-03,1,32.3182,-86.9023,state_centroid
4,1998-01-05,1,32.3182,-86.9023,state_centroid


In [22]:
### Load FARS data and filter for 6 target states
import pandas as pd
from pathlib import Path
import os

project_root = Path(os.getcwd()).parents[0]
data_folder = project_root / 'data' / 'data_accidents'

# Load full dataset (already with grid assignments)
df_crashes = pd.read_csv(data_folder / "fars_daily_hybrid_coords.csv", parse_dates=['date'])

# Define target states (FIPS codes)
TARGET_STATES = [6, 48, 12, 36, 17, 53]  # CA, TX, FL, NY, IL, WA

# Filter to only these states
df_subset = df_crashes[df_crashes['STATE'].isin(TARGET_STATES)].copy()

print(f"Filtered to {len(df_subset):,} crashes from {len(TARGET_STATES)} states")

Filtered to 303,425 crashes from 6 states


In [23]:
### Assign crashes to 1.0° grid (111km x 88km cells)
def assign_to_grid(df, grid_size=1.0):
    df = df.copy()
    df['grid_lat'] = (df['lat'] // grid_size) * grid_size + grid_size / 2
    df['grid_lon'] = (df['lon'] // grid_size) * grid_size + grid_size / 2
    df['grid_id'] = (
            df['grid_lat'].round(3).astype(str) + '_' +
            df['grid_lon'].round(3).astype(str)
    )
    return df


df_subset = assign_to_grid(df_subset, grid_size=1.0)

# Save filtered dataset
output_path = data_folder / "fars_daily_6states.csv"
df_subset.to_csv(output_path, index=False)
print(f"Saved {len(df_subset):,} crashes to {output_path}")
print(f"Total unique grid cells: {df_subset['grid_id'].nunique()}")  # Expect: 60–100

Saved 303,425 crashes to /Users/patricijamarijanovic/AVEIRO/FCD/foundations-of-data-science/data/data_accidents/fars_daily_6states.csv
Total unique grid cells: 251


In [25]:
### Fetch historical weather for just these 60–100 grid cells
import time
from tqdm import tqdm
import openmeteo_requests
import requests_cache
from retry_requests import retry

# Setup Open-Meteo client
cache_session = requests_cache.CachedSession('.cache', expire_after=86400)
retry_session = retry(cache_session, retries=3, backoff_factor=0.5)
openmeteo = openmeteo_requests.Client(session=retry_session)

grid_weather_dir = data_folder / "grid_weather_6states"
os.makedirs(grid_weather_dir, exist_ok=True)

# Get unique grid cells
unique_grids = df_subset[['grid_id', 'grid_lat', 'grid_lon']].drop_duplicates().reset_index(drop=True)
print(f"Fetching weather for {len(unique_grids)} grid cells...")

for _, row in tqdm(unique_grids.iterrows(), total=len(unique_grids), desc="Fetching weather"):
    grid_id = row['grid_id']
    lat = row['grid_lat']
    lon = row['grid_lon']
    output_path = grid_weather_dir / f"{grid_id}.parquet"

    # Skip if already downloaded
    if output_path.exists():
        continue

    try:
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": "1975-01-01",
            "end_date": "2023-12-31",
            "daily": ["temperature_2m_mean", "precipitation_sum", "wind_speed_10m_mean"],
            "timezone": "UTC"
        }

        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        daily = response.Daily()
        dates = pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left"
        )

        df_weather = pd.DataFrame({
            "date": dates.strftime('%Y-%m-%d'),
            "temperature_2m_mean": daily.Variables(0).ValuesAsNumpy(),
            "precipitation_sum": daily.Variables(1).ValuesAsNumpy(),
            "wind_speed_10m_mean": daily.Variables(2).ValuesAsNumpy(),
            "grid_id": grid_id
        })

        df_weather.to_parquet(output_path)

    except Exception as e:
        print(f"Error fetching {grid_id}: {e}")
        with open(grid_weather_dir / "failed_grids.txt", "a") as f:
            f.write(f"{grid_id},{lat},{lon},{str(e)}\n")


    time.sleep(8)

print("Weather data downloaded !")

Fetching weather for 251 grid cells...


Fetching weather:   2%|▏         | 4/251 [00:24<26:53,  6.53s/it]

Error fetching 31.5_-99.5: failed to request 'https://archive-api.open-meteo.com/v1/archive': {'reason': 'Minutely API request limit exceeded. Please try again in one minute.', 'error': True}


Fetching weather:   2%|▏         | 5/251 [00:32<28:47,  7.02s/it]

Error fetching 47.5_-120.5: failed to request 'https://archive-api.open-meteo.com/v1/archive': {'error': True, 'reason': 'Minutely API request limit exceeded. Please try again in one minute.'}


Fetching weather:   2%|▏         | 5/251 [00:34<28:36,  6.98s/it]


KeyboardInterrupt: 